In [705]:
import numpy as np
import random
import time
# Students will submit their files with their team-name.py
# Student have to use the Team as their parent class

Recent Version of the Code

In [706]:
class ROLLNUMBER_Q5:
  def __init__(self,num_balls,num_matches):
    self.num_balls = num_balls
    self.num_matches = num_matches
    self.num_arms = 6
    self.num_batters = 4
    self.wickets = np.zeros((self.num_batters, self.num_arms))
    self.occur = np.zeros((self.num_batters, self.num_arms))
    self.runs = np.zeros((self.num_batters, self.num_arms))
    self.prev_action = 0
    self.round = np.zeros(self.num_batters)
    self.order = np.array([0,1,2,3])
    self.prev_order = np.array([0,1,2,3])
    self.batter_idx = 3
    self.wickets_rem = 4
    self.epsilon = 1e-8
    self.ub = 1e-4
    self._hmap = np.array([1,1,2,3,4,6])
    self.hmap = np.array([0,1,2,3,4,6])
    self.start = 0
    self.a = np.ones((self.num_batters, self.num_arms))
    self.b = np.ones((self.num_batters, self.num_arms))
    self.a_w = np.ones((self.num_batters, self.num_arms))
    self.b_w = np.ones((self.num_batters, self.num_arms))
    
  @staticmethod
  def kl_div(p, q):
    if q==0 and p==0:
        return 0
    if q==0 and p!=0:
        return pow(10, 10)
    if q==1 and p==1:
        return 0
    if q==1 and p!=1:
        return pow(10, 10)
    if p==0:
        return np.log(1/(1-q))
    if p==1:
        return np.log(1/q)
    
    return p*np.log(p/q) + (1-p)*np.log((1-p)/(1-q))

  def kl_conf(self, t, em, num_pulls, precision=1e-6, max_iter=100):
    n = 0
    lower_bound = em
    upper_bound = 1
    
    while n<max_iter and (upper_bound-lower_bound)>precision:
        q = (lower_bound + upper_bound)/2
        if self.kl_div(em, q) < np.log(1+t*np.log(t)**2)/(num_pulls):
            upper_bound = q
        else:
            lower_bound = q
        n += 1
        
    return (upper_bound+lower_bound)/2
    
  def get_batting_order(self):
    return self.prev_order

  def get_action(self,wicket,runs_scored):
    X = wicket
    R = runs_scored
    player = self.prev_order[self.batter_idx]
    
    # Starter Code
    if self.start > 0:
        self.runs[player, self.prev_action] += R
        self.wickets[player, self.prev_action] += X
        self.occur[player, self.prev_action] += 1
        self.a_w[player, self.prev_action] += X
        self.b_w[player, self.prev_action] += 1-X
        
        if X==1:
            self.b[player, self.prev_action] += 1
        else:
            if(R==self.prev_action) or (R==6 and self.prev_action==5):
                self.a[player, self.prev_action] += 1
            else:
                self.b[player, self.prev_action] += 1
      
    # Updating Player after wicket fall
    if X == 1:
        self.wickets_rem -= 1
        self.batter_idx -= 1
        player = self.prev_order[self.batter_idx]
    
    # Re-assigning the variables after fall of 4 wickets
    if self.wickets_rem == 0:
        self.wickets_rem = 4
        self.batter_idx = 3
        
        ## Code to Implement for ordering
#         kl_conf_num = np.array([[self.kl_conf(self.round[j], self.runs[j, idx]/self._hmap[idx]/(self.occur[j, idx]), self.occur[j, idx]) for idx in range(6)] if self.round[j] > 6 else [0]*6 for j in range(self.num_batters)])
#         kl_conf_den = np.array([[self.kl_conf(self.round[j], (1 - self.wickets[j, idx]/self.occur[j, idx]), self.occur[j, idx]) for idx in range(6)] if self.round[j] > 6 else [0]*6 for j in range(self.num_batters)])
        kl_conf = np.array([[self.kl_conf(self.round[j], self.runs[j, idx]/self._hmap[idx]/(self.wickets[j, idx]+self.epsilon)*self.ub, self.occur[j, idx]) for idx in range(6)] if self.round[j] > 6 else [0]*6 for j in range(self.num_batters)])
        kl_max_conf = np.max(kl_conf, axis=1)
#         kl_max_conf = np.max(kl_conf_num/(1-kl_conf_den+self.epsilon), axis=1)
        self.prev_order = np.argsort(kl_max_conf)
#         thompson = np.random.beta(self.a, self.b)/np.random.beta(self.a_w, self.b_w)
#         thompson_max = np.max(thompson*self.hmap*np.random.beta(self.a, self.b)*self.hmap, axis=1)
#         self.prev_order = np.argsort(thompson_max)
        player = self.prev_order[self.batter_idx]
        
    # Inital Actions    
    if self.round[player] < 6:
        action = int(self.round[player])
        
    # Actions based on KL_UCB   
    else:
        kl_conf = np.array([self.kl_conf(self.round[player], (self.runs[player, idx]/self._hmap[idx])/(self.wickets[player, idx]+self.epsilon)*self.ub, self.occur[player, idx]) for idx in range(self.num_arms)])
        action = np.argmax(kl_conf*self.runs[player]*self.hmap/self.occur[player])
        thompson = np.random.beta(self.a[player], self.b[player])*self.hmap/np.random.beta(self.a_w[player], self.b_w[player])
        action = np.argmax(thompson*np.random.beta(self.a[player], self.b[player])*self.hmap)
        
    self.prev_action = action
    self.round[player] += 1
    self.start += 1
    
    return action



Psuedo Code for Question 5

def __init__(self, num_balls, num_matches):
    self.num_balls = num_balls
    self.num_matches = num_matches
    self.num_arms = 6
    self.num_batters = 4
    self.wickets = np.zeros((4,6))
    self.occur = np.zeros((4,6))
    self.runs = np.zeros((4,6))
    self.prev_action = 0
    self.round = np.zeros(4)
    self.prev_order = np.array([0,1,2,3])
    self.batter_idx = 3
    self.wickets_rem = 4
    self.epsilon = 1e-8
    self.ub = 1e-4
    self. _hmap = np.array([1,1,2,3,4,6])_.
    self.hmap = np.array([0,1,2,3,4,6])
    self.start = 0
    self.a = np.ones((4,6))
    self.b = np.ones((4,6))
    self.a_w = np.ones((4,6))
    self.b_w = np.ones((4,6))
    
    
Defining KL Div

Defining KL Conf


def get_batting_order(self):
    return self.prev_order
    
def get_action(self, wicket, runs):
    X = wicket
    R = runs
    player = self.prev_order[self.batter_idx]
    
    if self.start>0:
        self.runs[player, self.prev_action] += R
        self.wickets[player, self.prev_action += X
        self.occur[player, self.prev_action] += 1
        if X==1:
            self.a_w[player, self.prev_action] += 1
            self.b[player, self.prev_action] += 1
        else:
            self.b_w[player, self.prev_action] += 1
            if (R==self.prev_action) or (R==6 and self.prev_action==5):
                self.a[player, self.prev_action] += 1
            else:
                self.b[player, self.prev_action] += 1
    if X==1:
        self.wickets_rem -= 1
        self.batter_idx -= 1
        player = self.prev_order[self.batter_idx]
    
    if self.wickets_rem == 0:
        self.wickets_rem = 4
        self.batter_idx = 3
        
        #Kl Conf Logic here
        kl_conf = 
        kl_max_conf = np.max(kl_conf, axis=1)
        self.prev_order = np.argsort(kl_max_conf)
        player = self.prev_order[self.batter_idx]
        
    if self.round[player] < 6:
        action = self.round[player]
        
    else:
        # Thompson Logic here
        thompson = 
        action = np.argmax(thompson* ** Add S here ** *)
        
    self.prev_action = action
    self.round[player] += 1
    self.start += 1
    
    return action

In [807]:
class Environment:
  def __init__(self,num_balls,agent):
    self.num_balls = num_balls
    self.agent = agent
    self.__run_time = 0
    self.__total_runs = 0
    self.__total_wickets = 0
    self.__runs_scored = 0
    self.__start_time = 0
    self.__end_time = 0
    self.__p_out =np.array([[0.001,0.01,0.02,0.03,0.1,0.3] for i in range(4)])
    print(self.__p_out.shape)
    self.__p_run =np.array([[1,0.9,0.85,0.7,0.75,0.7] for i in range(3)]+[[0.2, 0.2, 0.2 ,0.2, 0.2, 0.2]])
    self.__action_runs_map = np.array([0,1,2,3,4,6])
    self.__wickets_left = 4
    self.__wicket = 0
    self.__runs_scored = 0
    self.__start_time = 0
    self.__end_time = 0
    self.__batting_order = np.array([0,1,2,3])
    self.__current_batter = self.__batting_order[self.__wickets_left-1]


  def __get_action(self):
    self.__start_time      = time. time()
    action          = self.agent.get_action(self.__wicket, self.__runs_scored)
    self.__end_time        = time. time()
    self.__run_time   = self.__run_time + self.__end_time - self.__start_time
    return action


  def __get_outcome(self, action):
    self.__current_batter = self.__batting_order[self.__wickets_left-1]
    pout = self.__p_out[self.__current_batter][action]
    prun = self.__p_run[self.__current_batter][action]
    wicket = np.random.choice(2,1,p=[1-pout,pout])[0]
    runs = 0
    if(wicket==0):
      runs = self.__action_runs_map[action]*np.random.choice(2,1,p=[1-prun,prun])[0]
    return wicket, runs


  def innings(self):
    self.__wickets_left = 4
    self.__runs_scored = 0
    self.__total_runs = 0
    self.__total_wickets = 0
    self.__run_time = 0
    self.__start_time = 0
    self.__end_time = 0


    self.__batting_order = self.agent.get_batting_order()

    for ball in range(self.num_balls):
      if (self.__wickets_left > 0 ) :
        action = self.__get_action()
        self.__wicket, self.__runs_scored   = self.__get_outcome(action)
        self.__total_runs     = self.__total_runs + self.__runs_scored
        if (self.__wicket > 0 ):
          self.__wickets_left = self.__wickets_left -1
        self.__total_wickets  = self.__total_wickets + self.__wicket
        if (self.__wickets_left == 0):
          self.__get_action()
    return self.__total_runs, self.__total_wickets, self.__run_time


In [808]:
class Prob5:
  def __init__(self,num_balls,num_matches,k=6):
    self.k = k
    self.muhats_run = np.zeros((4,self.k))
    self.muhats_wicket = np.zeros((4,self.k))
    self.epsilon = 1e-12
    self.action_map = [0,1,2,3,4,6]
    self.ucb_s = np.zeros((4,self.k))
    self.lcb_w = np.zeros((4,self.k)) + self.epsilon
    self.ucb_rho = self.ucb_s/self.lcb_w
    self.ucb_main = self.ucb_rho*self.ucb_s
    self.Ti = np.ones((4,self.k),dtype=int)
    self.t = np.zeros(4,dtype=int)
    self.prev = -1
    self.player_stats = np.zeros(4)
    self.matchcount = 0
    self.wicketcount = 0
    self.player = -1
    self.battingorder = None

  def d(self,p,q):
    if p == 0:
      return np.log(1/(1-q))
    elif p == 1:
      return np.log(1/q)
    return p*np.log(p/q) + (1-p)*np.log((1-p)/(1-q))

  def f(self,t):
    if t == 0:
      return 0
    return np.log(1 + t*(np.log(t)**2))

  def klucb(self,player,i):
    t = self.t[player]
    n = 0
    muhat = self.muhats_run[player,i]
    low = muhat
    high = 1
    max_iter = 50
    precision = 1e-9

    while n < max_iter and (high-low)>precision:
        q = (low+high)/2
        if self.d(muhat,q) > self.f(t)/self.Ti[player,i]:
            high = q
        else:
            low = q
        n += 1
    return (low+high)/2

  def kllcb(self,player,i):
    t = self.t[player]
    n = 0
    muhat = self.muhats_wicket[player,i]
    low = 0
    high = muhat
    max_iter = 50
    precision = 1e-3

    while n < max_iter and (high-low)>precision:
        q = (low+high)/2
        if self.d(muhat,q) < self.f(t)/self.Ti[player,i]:
            high = q
        else:
            low = q
        n += 1
    return (low+high)/2

  def update_ucbs_lcbs(self):
    for i in range(self.k):
      self.ucb_s[self.player,i] = self.action_map[i]*(self.klucb(self.player,i))
      self.lcb_w[self.player,i] = self.kllcb(self.player,i) + self.epsilon
      self.ucb_rho = self.ucb_s/self.lcb_w
      self.ucb_main = self.ucb_rho*self.ucb_s

  def get_action(self,wicket,runs_scored):
    self.update_player_stats()
    if wicket == 1:
      self.wicketcount += 1
      if self.wicketcount <= 4:
        self.player = self.battingorder[3-self.wicketcount]
    if self.wicketcount > 4:
      self.wicketcount = 0
    if self.prev > -1:
      t = self.Ti[self.player,self.prev]
      self.muhats_run[self.player,self.prev] = (self.muhats_run[self.player,self.prev]*(t-1) + int(runs_scored > 0))/t
      self.muhats_wicket[self.player,self.prev] = (self.muhats_wicket[self.player,self.prev]*(t-1) + wicket)/t

    if self.t[self.player] >= self.k:
      self.update_ucbs_lcbs()

    if self.t[self.player] < self.k:
      action = self.t[self.player]
    else:
      action = np.argmax(self.ucb_main[self.player,:])
      self.Ti[self.player,self.prev] += 1
    self.prev = action
    self.t[self.player] += 1
    print('Player: ',self.player,'Action: ',action,'Wicket Count: ',self.wicketcount,'Match Count: ',self.matchcount)
    return int(action)

  def update_player_stats(self):
    self.player_stats = np.max(self.ucb_main,axis=1)

  def get_batting_order(self):
    self.matchcount += 1
    self.battingorder = np.argsort(self.player_stats)
    self.player = self.battingorder[-1]
    print('New Match')
    print(self.player_stats)
    print(self.battingorder)
    return self.battingorder

In [860]:
num_matches = 100
num_balls = 60
agent = ROLLNUMBER_Q5(num_balls,num_matches)
# agent = Prob5(num_balls, num_matches)
environment = Environment(100,agent)
score = np.zeros((num_matches,1))
run_time = np.zeros((num_matches,1))
wicket = np.zeros((num_matches,1))
for i in range(num_matches):
    score[i],wicket[i],run_time[i] = environment.innings()
    score_hist.append(np.sum(score))

(4, 6)


In [861]:
# print(agent.battingorder)
print(agent.prev_order)

[3 1 0 2]


In [862]:
print(np.sum(score))

14422.0


In [791]:
score_hist = []
for idx in range(1191, 1231):
    np.random.seed(idx)
    num_matches = 20
    num_balls = 60
    agent = Prob5(num_balls, num_matches)
    environment = Environment(100,agent)
    score = np.zeros((num_matches,1))
    run_time = np.zeros((num_matches,1))
    wicket = np.zeros((num_matches,1))
    for i in range(num_matches):
      score[i],wicket[i],run_time[i] = environment.innings()
    score_hist.append(np.sum(score))

print(np.mean(score_hist))

(4, 6)
New Match
[0. 0. 0. 0.]
[0 1 2 3]
Player:  3 Action:  0 Wicket Count:  0 Match Count:  1
Player:  3 Action:  1 Wicket Count:  0 Match Count:  1
Player:  3 Action:  2 Wicket Count:  0 Match Count:  1
Player:  2 Action:  0 Wicket Count:  1 Match Count:  1
Player:  2 Action:  1 Wicket Count:  1 Match Count:  1
Player:  2 Action:  2 Wicket Count:  1 Match Count:  1
Player:  2 Action:  3 Wicket Count:  1 Match Count:  1
Player:  2 Action:  4 Wicket Count:  1 Match Count:  1
Player:  2 Action:  5 Wicket Count:  1 Match Count:  1
Player:  2 Action:  5 Wicket Count:  1 Match Count:  1
Player:  1 Action:  0 Wicket Count:  2 Match Count:  1
Player:  1 Action:  1 Wicket Count:  2 Match Count:  1
Player:  1 Action:  2 Wicket Count:  2 Match Count:  1
Player:  1 Action:  3 Wicket Count:  2 Match Count:  1
Player:  1 Action:  4 Wicket Count:  2 Match Count:  1
Player:  0 Action:  0 Wicket Count:  3 Match Count:  1
Player:  0 Action:  1 Wicket Count:  3 Match Count:  1
Player:  0 Action:  2 Wi

Player:  1 Action:  2 Wicket Count:  3 Match Count:  4
Player:  1 Action:  2 Wicket Count:  3 Match Count:  4
Player:  1 Action:  2 Wicket Count:  3 Match Count:  4
Player:  1 Action:  2 Wicket Count:  3 Match Count:  4
Player:  1 Action:  2 Wicket Count:  3 Match Count:  4
Player:  1 Action:  2 Wicket Count:  3 Match Count:  4
Player:  1 Action:  2 Wicket Count:  3 Match Count:  4
Player:  1 Action:  2 Wicket Count:  3 Match Count:  4
Player:  1 Action:  2 Wicket Count:  3 Match Count:  4
Player:  1 Action:  2 Wicket Count:  3 Match Count:  4
Player:  1 Action:  2 Wicket Count:  3 Match Count:  4
Player:  1 Action:  2 Wicket Count:  3 Match Count:  4
Player:  1 Action:  2 Wicket Count:  3 Match Count:  4
New Match
[6.22392835e+12 3.76415915e+12 3.17412153e+13 1.59733098e+13]
[1 0 3 2]
Player:  2 Action:  5 Wicket Count:  3 Match Count:  5
Player:  2 Action:  5 Wicket Count:  3 Match Count:  5
Player:  2 Action:  5 Wicket Count:  3 Match Count:  5
Player:  2 Action:  2 Wicket Count:  4

Player:  3 Action:  2 Wicket Count:  0 Match Count:  6
Player:  3 Action:  2 Wicket Count:  0 Match Count:  6
Player:  3 Action:  2 Wicket Count:  0 Match Count:  6
Player:  3 Action:  2 Wicket Count:  0 Match Count:  6
Player:  3 Action:  2 Wicket Count:  0 Match Count:  6
Player:  3 Action:  2 Wicket Count:  0 Match Count:  6
Player:  3 Action:  2 Wicket Count:  0 Match Count:  6
Player:  3 Action:  2 Wicket Count:  0 Match Count:  6
Player:  3 Action:  2 Wicket Count:  0 Match Count:  6
Player:  3 Action:  2 Wicket Count:  0 Match Count:  6
Player:  3 Action:  2 Wicket Count:  0 Match Count:  6
Player:  3 Action:  2 Wicket Count:  0 Match Count:  6
Player:  3 Action:  2 Wicket Count:  0 Match Count:  6
Player:  3 Action:  2 Wicket Count:  0 Match Count:  6
Player:  3 Action:  2 Wicket Count:  0 Match Count:  6
Player:  3 Action:  2 Wicket Count:  0 Match Count:  6
Player:  3 Action:  2 Wicket Count:  0 Match Count:  6
Player:  3 Action:  2 Wicket Count:  0 Match Count:  6
Player:  3

Player:  0 Action:  2 Wicket Count:  4 Match Count:  8
Player:  0 Action:  2 Wicket Count:  4 Match Count:  8
Player:  0 Action:  2 Wicket Count:  4 Match Count:  8
Player:  0 Action:  2 Wicket Count:  4 Match Count:  8
Player:  0 Action:  2 Wicket Count:  4 Match Count:  8
Player:  0 Action:  2 Wicket Count:  4 Match Count:  8
Player:  0 Action:  2 Wicket Count:  4 Match Count:  8
Player:  0 Action:  2 Wicket Count:  4 Match Count:  8
Player:  0 Action:  2 Wicket Count:  4 Match Count:  8
Player:  0 Action:  2 Wicket Count:  4 Match Count:  8
Player:  0 Action:  2 Wicket Count:  4 Match Count:  8
Player:  0 Action:  2 Wicket Count:  4 Match Count:  8
Player:  0 Action:  2 Wicket Count:  4 Match Count:  8
Player:  0 Action:  2 Wicket Count:  4 Match Count:  8
Player:  0 Action:  2 Wicket Count:  4 Match Count:  8
Player:  0 Action:  2 Wicket Count:  4 Match Count:  8
Player:  0 Action:  2 Wicket Count:  4 Match Count:  8
Player:  0 Action:  2 Wicket Count:  4 Match Count:  8
Player:  0

Player:  1 Action:  1 Wicket Count:  0 Match Count:  10
Player:  1 Action:  1 Wicket Count:  0 Match Count:  10
Player:  1 Action:  1 Wicket Count:  0 Match Count:  10
Player:  1 Action:  1 Wicket Count:  0 Match Count:  10
Player:  1 Action:  1 Wicket Count:  0 Match Count:  10
Player:  1 Action:  1 Wicket Count:  0 Match Count:  10
Player:  1 Action:  1 Wicket Count:  0 Match Count:  10
Player:  1 Action:  1 Wicket Count:  0 Match Count:  10
Player:  1 Action:  1 Wicket Count:  0 Match Count:  10
Player:  1 Action:  1 Wicket Count:  0 Match Count:  10
Player:  1 Action:  1 Wicket Count:  0 Match Count:  10
Player:  1 Action:  1 Wicket Count:  0 Match Count:  10
Player:  1 Action:  1 Wicket Count:  0 Match Count:  10
Player:  1 Action:  1 Wicket Count:  0 Match Count:  10
Player:  1 Action:  1 Wicket Count:  0 Match Count:  10
Player:  1 Action:  1 Wicket Count:  0 Match Count:  10
Player:  1 Action:  1 Wicket Count:  0 Match Count:  10
Player:  1 Action:  1 Wicket Count:  0 Match Cou

...

Player:  3 Action:  4 Wicket Count:  2 Match Count:  19
Player:  3 Action:  4 Wicket Count:  2 Match Count:  19
Player:  3 Action:  4 Wicket Count:  2 Match Count:  19
Player:  3 Action:  4 Wicket Count:  2 Match Count:  19
Player:  3 Action:  4 Wicket Count:  2 Match Count:  19
Player:  3 Action:  4 Wicket Count:  2 Match Count:  19
Player:  3 Action:  4 Wicket Count:  2 Match Count:  19
Player:  3 Action:  4 Wicket Count:  2 Match Count:  19
Player:  3 Action:  4 Wicket Count:  2 Match Count:  19
Player:  3 Action:  4 Wicket Count:  2 Match Count:  19
Player:  3 Action:  4 Wicket Count:  2 Match Count:  19
Player:  3 Action:  4 Wicket Count:  2 Match Count:  19
Player:  3 Action:  4 Wicket Count:  2 Match Count:  19
Player:  3 Action:  4 Wicket Count:  2 Match Count:  19
Player:  3 Action:  4 Wicket Count:  2 Match Count:  19
Player:  3 Action:  4 Wicket Count:  2 Match Count:  19
Player:  3 Action:  4 Wicket Count:  2 Match Count:  19
Player:  3 Action:  4 Wicket Count:  2 Match Cou

Player:  3 Action:  3 Wicket Count:  0 Match Count:  4
Player:  3 Action:  3 Wicket Count:  0 Match Count:  4
Player:  3 Action:  3 Wicket Count:  0 Match Count:  4
Player:  3 Action:  3 Wicket Count:  0 Match Count:  4
Player:  3 Action:  3 Wicket Count:  0 Match Count:  4
Player:  3 Action:  3 Wicket Count:  0 Match Count:  4
Player:  3 Action:  3 Wicket Count:  0 Match Count:  4
Player:  3 Action:  3 Wicket Count:  0 Match Count:  4
Player:  3 Action:  3 Wicket Count:  0 Match Count:  4
Player:  3 Action:  3 Wicket Count:  0 Match Count:  4
Player:  3 Action:  3 Wicket Count:  0 Match Count:  4
Player:  3 Action:  3 Wicket Count:  0 Match Count:  4
Player:  3 Action:  3 Wicket Count:  0 Match Count:  4
Player:  3 Action:  3 Wicket Count:  0 Match Count:  4
Player:  3 Action:  3 Wicket Count:  0 Match Count:  4
Player:  3 Action:  3 Wicket Count:  0 Match Count:  4
Player:  1 Action:  5 Wicket Count:  1 Match Count:  4
Player:  1 Action:  5 Wicket Count:  1 Match Count:  4
Player:  0

Player:  1 Action:  1 Wicket Count:  4 Match Count:  6
Player:  1 Action:  1 Wicket Count:  4 Match Count:  6
Player:  1 Action:  1 Wicket Count:  4 Match Count:  6
Player:  1 Action:  1 Wicket Count:  4 Match Count:  6
Player:  1 Action:  1 Wicket Count:  4 Match Count:  6
Player:  1 Action:  1 Wicket Count:  4 Match Count:  6
Player:  1 Action:  1 Wicket Count:  4 Match Count:  6
Player:  1 Action:  1 Wicket Count:  4 Match Count:  6
Player:  1 Action:  1 Wicket Count:  4 Match Count:  6
Player:  1 Action:  1 Wicket Count:  4 Match Count:  6
Player:  1 Action:  1 Wicket Count:  4 Match Count:  6
Player:  1 Action:  1 Wicket Count:  4 Match Count:  6
Player:  1 Action:  1 Wicket Count:  4 Match Count:  6
Player:  1 Action:  1 Wicket Count:  4 Match Count:  6
Player:  1 Action:  1 Wicket Count:  4 Match Count:  6
Player:  1 Action:  1 Wicket Count:  4 Match Count:  6
Player:  1 Action:  1 Wicket Count:  4 Match Count:  6
Player:  1 Action:  1 Wicket Count:  4 Match Count:  6
Player:  1

Player:  0 Action:  3 Wicket Count:  4 Match Count:  8
Player:  0 Action:  3 Wicket Count:  4 Match Count:  8
Player:  0 Action:  2 Wicket Count:  0 Match Count:  8
Player:  0 Action:  2 Wicket Count:  0 Match Count:  8
Player:  0 Action:  2 Wicket Count:  0 Match Count:  8
Player:  0 Action:  2 Wicket Count:  0 Match Count:  8
Player:  0 Action:  2 Wicket Count:  0 Match Count:  8
Player:  0 Action:  2 Wicket Count:  0 Match Count:  8
Player:  0 Action:  2 Wicket Count:  0 Match Count:  8
Player:  0 Action:  2 Wicket Count:  0 Match Count:  8
Player:  0 Action:  2 Wicket Count:  0 Match Count:  8
Player:  0 Action:  2 Wicket Count:  0 Match Count:  8
Player:  0 Action:  2 Wicket Count:  0 Match Count:  8
Player:  0 Action:  2 Wicket Count:  0 Match Count:  8
Player:  0 Action:  2 Wicket Count:  0 Match Count:  8
Player:  0 Action:  2 Wicket Count:  0 Match Count:  8
Player:  0 Action:  2 Wicket Count:  0 Match Count:  8
Player:  0 Action:  2 Wicket Count:  0 Match Count:  8
Player:  0

KeyboardInterrupt: 

In [700]:
np.sum(score)

621.0

In [600]:
arr = np.random.randn(4,6)

In [335]:
np.max(arr, axis=1)

array([1.38799141, 1.42855189, 1.20726784, 1.60173494])

In [333]:
np.argsort(np.max(arr, axis=1))

array([2, 0, 1, 3], dtype=int64)

In [420]:
a = np.ones((4,6))

In [422]:
a*np.array([0,1,2,3,4,6])

array([[0., 1., 2., 3., 4., 6.],
       [0., 1., 2., 3., 4., 6.],
       [0., 1., 2., 3., 4., 6.],
       [0., 1., 2., 3., 4., 6.]])